# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [36]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [37]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [38]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [39]:
foldr(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [40]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату.

In [41]:
#def foldl2(f, x0, lst):
#    return foldr(..., ..., lst)(...)

In [42]:
#def foldr2(f, x0, lst):
#    return foldl(..., ..., lst)(...)

In [43]:
def foldl2(f, x0, lst):
    return foldr(lambda val, func: lambda x: func(f(x, val)), lambda base: base, lst)(x0)

In [44]:
def foldr2(f, x0, lst):
    return foldl(lambda func, val: lambda x: func(f(val, x)), lambda base: base, lst)(x0)

In [45]:
foldl2(lambda x, y: x + y, 0, [1, 2, 3])

6

In [46]:
foldr2(lambda x, y: x + y, 0, [1, 2, 3])

6

In [47]:
foldl2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [48]:
foldr2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [49]:
f = lambda x, y: x / y
foldl2(f, 1, [1, 2, 3]), \
foldr2(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [50]:
from collections import Counter

def is_anagram(a, b):
    if Counter(b) == Counter(a):
        return True
    else:
        return False
    
def check_inv(a,b):
    for i in range(len(a) - len(b) + 1):
        if is_anagram(a[i:i+len(b)],b):
            return True
        
a = 'abcrotm'
b = 'tro'
check_inv(a,b)

#Сложность O(m*n)

True

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [51]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if self.left == None and self.right == None:
            yield self.value
        if self.left:
            yield from self.left
        if self.right:
            yield from self.right
    
    def __str__(self):
        return str([self.value, self.left, self.right])
    
    def __repr__(self):
        return f'(NODE {self.value}, LEFT {repr(self.left)}, RIGHT {repr(self.right)})'
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))

list(tree) == [3, 4, 2]

True

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [52]:
import re

operations = {
  "+": (lambda a, b: a + b),
  "-": (lambda a, b: a - b),
  "*": (lambda a, b: a * b),
  "/": (lambda a, b: a / b)
}

def oper (s, op):
    res = list(s)
    for i in range(len(s)):
        if s[i] == op:
            left_idx = i - 1
            left = ''
            while s[left_idx] in '0123456789':
                left = s[left_idx] + left
                left_idx -= 1
                if left_idx < 0:
                    break
            right_idx = i + 1
            left_idx += 1
            right = ''
            while s[right_idx] in '0123456789':
                right = right + s[right_idx]
                right_idx += 1
                if right_idx == len(s):
                    break
            tmp = operations[op](int(left),int(right))
            for i in range(len(str(tmp))):
                res[left_idx + i] = str(tmp)[i]
            for i in range(left_idx + len(str(tmp)), right_idx):
                res[i] = ''
    return "".join(res)
        
def calc(s):
    s = re.sub(r'[^-0-9()*+/]', '', s)
    res = re.findall(r'[(][-0-9*+/]+', s)
    tmp = []
    for i in res:
        tmp.append(oper(oper(oper(oper(i[1:], '*'), '/'), '-'), '+'))
    for i in range(len(res)):
        while s.find((res[i]+')')) > 0:
            j = s.find((res[i]+')'))
            s = s[:j] + tmp[i] + s[j+len(res[i]+')'):]
    return oper(oper(oper(oper(s, '*'), '/'), '-'), '+')

calc('2 * (15 - 3 * 4) - 2') == '4'

True